In [118]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.append('../InputData/')
from ParsePaiBrandaosFiles import read_input_files
plt.style.use('../styles/mpl2dark.mplstyle')
import seaborn as sns
from keras.models import Sequential
from keras.layers import Dense
#from keras.layers import ReLu
import keras.optimizers
import copy

In [ ]:
#this is a direct translation from fortran of my father's cost function 
def fdisttraj(traj1,traj2,w):
  # traj1(48), traj2(48), w(48), wl(48)
  
  wl=w
  wl[37]=w[37]/10.0  # more weight for the inpact parameter
  wl[46]=w[46]/10.0  # more weight for the collision energy
  dist=0.0
  for i in range(48):
    dist=dist+((traj1[i]-traj2[i])/wl[i])**2
  
  return np.sqrt(dist)

In [128]:
#this is a first draft of the function where we can test different parameters for the network
# currently I am only changing the number of layers, but we should be changing more stuff... 
# including those already in the function arguments and the optimization parameters, which are currently hard coded
def fine_tune_model(keras_model, x_train, y_train, activation_function, min_depth, max_depth, min_breadth, max_breadth):    
    trained_models = []
    for depth in range(min_depth,max_depth):
        keras_model_trial = Sequential();#keras_model.deepcopy()
        [keras_model_trial.add(Dense(units=100, activation='relu', input_dim=24)) for dep in range(depth)]
        keras_model_trial.add(Dense(units=24, activation='linear'))
        #[keras_model_trial.add(Dense(units=100, activation='relu', input_dim=24)) for dep in range(depth)]
        #keras_model_trial.add(Dense(units=24, activation='tanh'))
        keras_model_trial.compile(loss='mean_squared_error',
              optimizer=keras.optimizers.SGD(lr=0.0001, momentum=0.9, nesterov=True),
              metrics=['accuracy'])
        keras_model_trial.fit(x_train,y_train, epochs=50, batch_size=10)
        keras_model_trial.test_on_batch(x_train,y_train)
        trained_models.append(keras_model_trial)
    return trained_models


In [17]:
df = read_input_files("../data/h2h2_10K.ver")#.sample(frac=0.01, replace=False, random_state=1)

In [14]:
x_train = df[['xi1', 'yi1', 'zi1', 'xi2', 'yi2', 'zi2', 'pxi1', 'pyi1', 'pzi1',
       'pxi2', 'pyi2', 'pzi2', 'xi3', 'yi3', 'zi3', 'xi4', 'yi4', 'zi4',
       'pxi3', 'pyi3', 'pzi3', 'pxi4', 'pyi4', 'pzi4']].values
y_train = df[['xf1', 'yf1', 'zf1',
       'xf2', 'yf2', 'zf2', 'pxf1', 'pyf1', 'pzf1', 'pxf2', 'pyf2', 'pzf2',
       'xf3', 'yf3', 'zf3', 'xf4', 'yf4', 'zf4', 'pxf3', 'pyf3', 'pzf3',
       'pxf4', 'pyf4', 'pzf4']].values

In [129]:
keras_model = Sequential()
fine_tune_model(keras_model,x_train,y_train,0,1,6,0,0)

Epoch 1/50
4540/4540 [==============================] - 0s 40us/step - loss: 2.9138 - acc: 0.2167
Epoch 2/50
4540/4540 [==============================] - 0s 42us/step - loss: 2.1111 - acc: 0.2280
Epoch 3/50
4540/4540 [==============================] - 0s 43us/step - loss: 1.9571 - acc: 0.2348
Epoch 4/50
4540/4540 [==============================] - 0s 42us/step - loss: 1.8670 - acc: 0.2476
Epoch 5/50
4540/4540 [==============================] - 0s 42us/step - loss: 1.8053 - acc: 0.2562
Epoch 6/50
4540/4540 [==============================] - 0s 39us/step - loss: 1.7530 - acc: 0.2623
Epoch 7/50
4540/4540 [==============================] - 0s 42us/step - loss: 1.7034 - acc: 0.2632
Epoch 8/50
4540/4540 [==============================] - 0s 42us/step - loss: 1.6535 - acc: 0.2685
Epoch 9/50
4540/4540 [==============================] - 0s 39us/step - loss: 1.6050 - acc: 0.2714
Epoch 10/50
4540/4540 [==============================] - 0s 42us/step - loss: 1.5597 - acc: 0.2859
Epoch 11/50
4540/45

In [124]:
models = fine_tune_model(keras_model,x_train,y_train,0,5,6,0,0)

Epoch 1/5
4540/4540 [==============================] - 0s 80us/step - loss: 3.6482 - acc: 0.1989
Epoch 2/5
4540/4540 [==============================] - 0s 97us/step - loss: 2.0952 - acc: 0.2368
Epoch 3/5
4540/4540 [==============================] - 1s 126us/step - loss: 1.7820 - acc: 0.2511
Epoch 4/5
4540/4540 [==============================] - 1s 131us/step - loss: 1.5902 - acc: 0.2610
Epoch 5/5
4540/4540 [==============================] - 1s 128us/step - loss: 1.4541 - acc: 0.3026


In [126]:
y = models[-1].predict(x_train[:2])

In [127]:
y[0]

array([-0.08715117,  1.0431813 ,  6.9414854 ,  0.04016537,  1.1324009 ,
        6.833683  ,  0.01288999,  5.97669   , -1.1666867 , -0.04679969,
        5.9957304 , -1.0495169 ,  0.01562551, -0.0319979 , -0.0629486 ,
        0.09171814,  0.07975957,  0.10899325,  0.07904167,  0.04934346,
        0.04786381,  0.04713698,  0.04975314,  0.07107562], dtype=float32)

In [113]:
y_train[0]

array([-2.00200e-03, -2.69000e-03,  6.93198e+00,  4.25040e-02,
        6.37464e-01,  6.74654e+00, -1.57205e-01,  6.30655e+00,
       -1.29815e+00,  1.16704e-01,  6.98705e+00, -9.78602e-01,
        2.47240e-02,  3.71651e-01, -9.18070e-02, -2.44920e-02,
       -3.68091e-01,  1.26648e-01,  1.55677e-01,  3.88580e-01,
        1.63806e-01, -1.55909e-01, -3.92140e-01, -1.98647e-01])

In [49]:
y[2]

array([6.9323152e-09, 8.7908660e-07, 3.0128226e-01, 1.6075105e-04,
       6.5488109e-07, 2.9816747e-01, 8.8253571e-04, 1.9161262e-01,
       3.7180458e-04, 1.9027153e-06, 2.0274951e-01, 4.1941978e-04,
       7.3901980e-08, 5.4235937e-08, 1.1710284e-03, 2.9173796e-07,
       4.5212227e-04, 2.6737666e-03, 7.5531887e-08, 3.5582150e-05,
       3.9415777e-08, 1.7122888e-05, 3.5943788e-08, 3.8733536e-09],
      dtype=float32)